# 2022 NHL Playoff Pool

In [1]:
import pandas as pd
import numpy as np
import os

In [21]:
# Initialize dicts
picks = {}
entries = {}
points = {}

In [3]:
brackets = os.listdir('brackets')

In [4]:
def read_brackets_loop(brackets):
    """Read playoff bracket entries and save results as dict"""
    for bracket in brackets:
        df = pd.read_excel("brackets/"+bracket, sheet_name='teams')
        entrant_name = df['entrant_name'].dropna().item()
        if entrant_name in picks.keys():
            entrant_name+=' 2' 
        picks[entrant_name] = {
            'rd1_selections' : df['rd1_selections'].dropna().to_list(),
            'rd2_selections' : df['rd2_selections'].dropna().to_list(),
            'rd3_selections' : df['rd3_selections'].dropna().to_list(),
            'rd4_selections' : df['rd4_selections'].dropna().to_list(),
            'tie_breaker' : df['tie_breaker'].dropna().to_list()
        }
    return picks

In [5]:
read_brackets_loop(brackets)

{'Cameron Harris': {'rd1_selections': ['NSH',
   'EDM',
   'COL',
   'MIN',
   'FLA',
   'TBL',
   'CAR',
   'NYR'],
  'rd2_selections': ['EDM', 'MIN', 'FLA', 'CAR'],
  'rd3_selections': ['EDM', 'FLA'],
  'rd4_selections': ['FLA'],
  'tie_breaker': ['Pasternak']},
 'Cameron Harris 2': {'rd1_selections': ['CGY',
   'EDM',
   'COL',
   'MIN',
   'FLA',
   'TOR',
   'BOS',
   'NYR'],
  'rd2_selections': ['CGY', 'COL', 'FLA', 'BOS'],
  'rd3_selections': ['COL', 'BOS'],
  'rd4_selections': ['COL'],
  'tie_breaker': ['Pasternak']}}

In [65]:
# Attempt class method to make this cleaner
class bracket_entry:
    """Class to manage bracket entries and calculate points"""
    def __init__(self, bracket_name):
        self.bracket_name = bracket_name
        self.picks = {}
        self.points = {}
        
    def read_brackets(self):
        """Read playoff bracket entries and save results as dict"""
        directory = "brackets/" + self.bracket_name
        df = pd.read_excel(directory, sheet_name='teams')
        self.entrant_name = df['entrant_name'].dropna().item()
        if self.entrant_name in picks.keys():
            self.entrant_name = self.entrant_name + ' 2'

        self.picks[self.entrant_name] = {
            'rd1_selections' : df['rd1_selections'].dropna().to_list(),
            'rd2_selections' : df['rd2_selections'].dropna().to_list(),
            'rd3_selections' : df['rd3_selections'].dropna().to_list(),
            'rd4_selections' : df['rd4_selections'].dropna().to_list(),
            'tie_breaker' : df['tie_breaker'].dropna().to_list()
        }
        return
    
    def calc_points(self, results):
        """Calculate pool points based on latest playoff results"""
        points_by_round = {
            'rd1': 2,
            'rd2': 4,
            'rd3': 8,
            'rd4': 17
        }
        for rd in results.keys():
            round_picks = self.picks[self.entrant_name][rd+'_selections']
            round_result = results[rd]
            round_points = len(set(round_picks) & set(round_result))*points_by_round[rd]
            self.points[rd+'_points'] = round_points
        return

In [66]:
# Initialize entries
for n, bracket in enumerate(brackets):
    print(bracket)
    entries[str(n+1)] = bracket_entry(bracket)

bracket_2022_CameronTest1.xlsx
bracket_2022_CameronTest.xlsx


In [67]:
entries['1'].bracket_name

'bracket_2022_CameronTest1.xlsx'

In [68]:
entries['1'].picks

{}

In [69]:
entries['1'].points

{}

In [70]:
# Extract picks
for entry_num, entry in entries.items():
    bracket_entry.read_brackets(entry)

In [71]:
entries['1'].picks

{'Cameron Harris': {'rd1_selections': ['NSH',
   'EDM',
   'COL',
   'MIN',
   'FLA',
   'TBL',
   'CAR',
   'NYR'],
  'rd2_selections': ['EDM', 'MIN', 'FLA', 'CAR'],
  'rd3_selections': ['EDM', 'FLA'],
  'rd4_selections': ['FLA'],
  'tie_breaker': ['Pasternak']}}

In [72]:
entries['2'].picks

{'Cameron Harris': {'rd1_selections': ['CGY',
   'EDM',
   'COL',
   'MIN',
   'FLA',
   'TOR',
   'BOS',
   'NYR'],
  'rd2_selections': ['CGY', 'COL', 'FLA', 'BOS'],
  'rd3_selections': ['COL', 'BOS'],
  'rd4_selections': ['COL'],
  'tie_breaker': ['Pasternak']}}

In [73]:
pd.DataFrame(entries['2'].picks)

,Cameron Harris
rd1_selections,"[CGY, EDM, COL, MIN, FLA, TOR, BOS, NYR]"
rd2_selections,"[CGY, COL, FLA, BOS]"
rd3_selections,"[COL, BOS]"
rd4_selections,[COL]
tie_breaker,[Pasternak]


In [74]:
# Results by round
results = {
    'rd1': [
        'CGY',
        'EDM',
        'COL',
        'MIN',
        'FLA',
        'TOR',
        'CAR',
        'PIT'
    ],
    'rd2': [
        'EDM',
        'COL',
        'FLA',
        'PIT'
    ],
    'rd3': [
        'COL',
        'FLA'
    ],
    'rd4': [
        'FLA'
    ]
}

In [75]:
results.keys()

dict_keys(['rd1', 'rd2', 'rd3', 'rd4'])

In [76]:
# Calculate points based on latest results
for entry_num, entry in entries.items():
    bracket_entry.calc_points(entry, results)

In [78]:
entries['1'].points

{'rd1_points': 10, 'rd2_points': 8, 'rd3_points': 8, 'rd4_points': 17}

## TO DO
Investigate why this only works when I assign output from class method to a new dict, thougth I should be able to save the picks/results as attributes of each instance (e.g. self.picks, self.results)

In [ ]:
points

In [ ]:
points_df = pd.DataFrame(points).T
points_df['total'] = points_df.iloc[:, :].sum(axis=1)
points_df

In [ ]:
points_df.to_csv("point_summary.csv")